In [1]:
%run utils.ipynb
random.seed(0)

In [2]:
root_dir = "./files/vehicle_models"

In [3]:
class_dirs = [os.path.join(root_dir, i) for i in os.listdir(root_dir)]
random.shuffle(class_dirs)
print(f"{len(class_dirs)} classes")

160 classes


In [4]:
class_dirs = class_dirs[:80]

In [5]:
x_train, y_train = [], []
x_test, y_test = [], []

In [6]:
%%time
for class_label, class_dir in tqdm(enumerate(class_dirs)):
    train_list, test_list = list_split(get_all_images(class_dir, shuffle = False),
                                       ratio = [0.6, 0.4], shuffle = False)
    for image_path in train_list:    
        _, _, roi = roi_extraction_1(image_path)
        roi = resize(roi, is_gray = True, width = 320, height = 160)
        feature = cvhog.compute(roi)
        x_train.append(feature)
        y_train.append(class_label)
 
    for image_path in test_list:    
        _, _, roi = roi_extraction_1(image_path)
        roi = resize(roi, is_gray = True, width = 320, height = 160)
        feature = cvhog.compute(roi)
        x_test.append(feature)
        y_test.append(class_label)


CPU times: user 1min 14s, sys: 1.72 s, total: 1min 16s
Wall time: 1min 7s


In [7]:
f_dim = x_test[0].shape[0]
print(f"feature dim : {f_dim}")

feature dim : 3780


In [8]:
x_train = np.asarray(x_train).reshape(-1, f_dim)
x_test = np.asarray(x_test).reshape(-1, f_dim)

In [9]:
x_train.shape

(1447, 3780)

In [10]:
%%time
# Grid Search
# Parameter Grid
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001, 0.00001, 10]}
 
# Make grid search classifier
clf_grid = GridSearchCV(svm.SVC(), param_grid, verbose=1, pre_dispatch = cpu_count//2)
 
# Train the classifier
clf_grid.fit(x_train, y_train)
 
# clf = grid.best_estimator_()
print("Best Parameters:\n", clf_grid.best_params_)
print("Best Estimators:\n", clf_grid.best_estimator_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 20.6min finished


Best Parameters:
 {'C': 100, 'gamma': 0.001}
Best Estimators:
 SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
CPU times: user 20min 44s, sys: 414 ms, total: 20min 44s
Wall time: 20min 44s


In [11]:
# Make predictions on unseen test data
y_pred = clf_grid.predict(x_test)
print("Accuracy: {}%".format(clf_grid.score(x_test, y_test) * 100 ))

Accuracy: 97.32510288065843%


In [12]:
overall_acc = overall_accuracy(y_test, y_pred)
m_prec = mean_prec(y_test, y_pred)
m_recall = mean_recall(y_test, y_pred)

print(f"Over all acc : {overall_acc*100:.2f}%\nmean prec : {m_prec*100:.2f}%\nmean recall : {m_recall*100:.2f}%")

Over all acc : 97.33%
mean prec : 97.64%
mean recall : 95.97%


- roi plan 1